# LSTM/GRU高级应用：时间序列预测

本notebook展示了如何使用LSTM和GRU模型进行时间序列预测，以Jena气候数据集为例。

## 1. 数据加载与初步分析

In [ ]:
import os
import numpy as np
import tensorflow as tf
from pathlib import Path

# 设置全局随机种子以确保结果可复现
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

# 数据路径配置 - 请根据实际情况修改
DATA_DIR = Path.home() / "下载" / "jena_climate"
DATA_FILE = DATA_DIR / "jena_climate_2009_2016.csv"

# 检查数据文件是否存在
if not DATA_FILE.exists():
    raise FileNotFoundError(
        f"数据文件未找到: {DATA_FILE}\n"
        f"请下载Jena气候数据集并放置在正确的位置。\n"
        f"下载地址: https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip"
    )

# 读取数据文件
with open(DATA_FILE, 'r', encoding='utf-8') as f:
    data = f.read()

# 解析CSV数据
lines = data.split("\n")
header = lines[0].split(",")
lines = lines[1:]

print(f"数据列名: {header}")
print(f"数据记录数: {len(lines)}")
print(f"采样频率: 每10分钟一次")
print(f"时间跨度: 2009-2016年，约{len(lines) / (6 * 24 * 365):.1f}年")

## 2. 数据解析与预处理

将原始CSV数据转换为NumPy数组，方便后续处理。

In [ ]:
# 将数据转换为浮点数数组
# 跳过第一列（日期时间），只保留数值列
float_data = np.zeros((len(lines), len(header) - 1))

for i, line in enumerate(lines):
    # 分割每行，跳过日期时间列，提取数值
    values = [float(x) for x in line.split(",")[1:]]
    float_data[i, :] = values

print(f"数据矩阵形状: {float_data.shape}")
print(f"特征数量: {float_data.shape[1]}")
print(f"时间步数: {float_data.shape[0]}")

## 3. 数据可视化分析

可视化温度变化趋势，帮助理解数据特征。

In [ ]:
import matplotlib.pyplot as plt

# 温度数据在第2列（索引为1）
temp = float_data[:, 1]

# 绘制完整时间序列
plt.figure(figsize=(15, 5))
plt.plot(range(len(temp)), temp, linewidth=0.5)
plt.title("Temperature Time Series (2009-2016)", fontsize=14, pad=15)
plt.xlabel("Time Steps (10-minute intervals)", fontsize=12)
plt.ylabel("Temperature (°C)", fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# 绘制前10天的温度数据（1天 = 6 * 24 = 144个时间步）
# 10天 = 1440个时间步
plt.figure(figsize=(15, 5))
plt.plot(range(1440), temp[:1440], linewidth=1.5)
plt.title("Temperature Time Series (First 10 Days)", fontsize=14, pad=15)
plt.xlabel("Time Steps (10-minute intervals)", fontsize=12)
plt.ylabel("Temperature (°C)", fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. 数据标准化

使用Z-score标准化方法对数据进行归一化处理。

**注意**: 
- 只使用训练集（前200,000个样本）的统计量进行标准化
- 这样可以避免数据泄露，确保模型不会利用验证集和测试集的信息

In [ ]:
# 使用训练集的统计量进行标准化
TRAIN_SIZE = 200000
mean = float_data[:TRAIN_SIZE].mean(axis=0)
std = float_data[:TRAIN_SIZE].std(axis=0)

# 标准化整个数据集
float_data -= mean
float_data /= std

print(f"训练集大小: {TRAIN_SIZE}")
print(f"均值: {mean[:3]}...")  # 显示前3个特征的均值
print(f"标准差: {std[:3]}...")  # 显示前3个特征的标准差

## 5. 时间序列数据生成器

实现一个高效的数据生成器，用于按批次生成时间序列样本。

**生成器的作用**：
- 避免一次性将所有数据加载到内存
- 支持数据增强（如随机采样）
- 提高训练效率

In [ ]:
def generator(data, lookback, delay, min_index, max_index, 
              shuffle=False, batch_size=128, step=6):
    """
    时间序列数据生成器
    
    Parameters:
    -----------
    data : ndarray
        完整的时间序列数据，形状为 (timesteps, features)
    lookback : int
        输入序列的时间窗口长度（回溯多少个时间步）
    delay : int
        目标值相对于当前时间的延迟（预测未来多少个时间步）
    min_index : int
        数据索引的最小值（用于划分训练/验证/测试集）
    max_index : int or None
        数据索引的最大值，None表示使用数据末尾
    shuffle : bool
        是否随机采样（训练时True，验证/测试时False）
    batch_size : int
        每批次的样本数量
    step : int
        时间步采样间隔（用于降采样，节省计算资源）
        
    Yields:
    -------
    samples : ndarray
        输入序列，形状为 (batch_size, lookback//step, features)
    targets : ndarray
        目标值，形状为 (batch_size,)
    """
    if max_index is None:
        max_index = len(data) - delay - 1
    
    i = min_index + lookback
    
    while True:
        if shuffle:
            # 训练时随机采样
            rows = np.random.randint(min_index + lookback, max_index, size=batch_size)
        else:
            # 验证/测试时顺序采样
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)
        
        # 初始化样本和目标数组
        samples = np.zeros((len(rows), lookback // step, data.shape[-1]))
        targets = np.zeros((len(rows),))
        
        # 填充数据
        for j, row in enumerate(rows):
            indices = range(row - lookback, row, step)
            samples[j] = data[indices]
            targets[j] = data[row + delay][1]  # 预测温度（第2列）
        
        yield samples, targets

## 6. 创建训练、验证和测试数据生成器

**数据集划分**：
- 训练集: 0 - 200,000 (约4.75年)
- 验证集: 200,001 - 300,000 (约2.38年)
- 测试集: 300,001 - 结束 (约2.87年)

**预测任务**：
- 使用过去10天的数据（lookback=1440，即144×10个时间步）
- 预测未来24小时后的温度（delay=144，即24小时）
- 采样间隔为1小时（step=6，即每小时采样一次）

In [ ]:
# 配置参数
lookback = 1440  # 回溯1440个时间步（10天）
delay = 144      # 预测144个时间步后的温度（24小时后）
step = 6         # 采样间隔为6（每小时采样一次）
batch_size = 128

# 创建训练数据生成器（使用随机采样）
train_gen = generator(
    float_data,
    lookback=lookback,
    delay=delay,
    min_index=0,
    max_index=200000,
    shuffle=True,
    step=step,
    batch_size=batch_size
)

# 创建验证数据生成器（顺序采样）
val_gen = generator(
    float_data,
    lookback=lookback,
    delay=delay,
    min_index=200001,
    max_index=300000,
    step=step,
    batch_size=batch_size
)

# 创建测试数据生成器（顺序采样）
test_gen = generator(
    float_data,
    lookback=lookback,
    delay=delay,
    min_index=300001,
    max_index=None,
    step=step,
    batch_size=batch_size
)

# 计算验证和测试所需的步数
val_steps = (300000 - 200001 - lookback) // batch_size
test_steps = (len(float_data) - 300001 - lookback) // batch_size

print(f"验证步数: {val_steps}")
print(f"测试步数: {test_steps}")

## 7. 建立性能基准

在训练复杂模型之前，先建立一个简单的基准模型。

**朴素方法（Naive Method）**：
- 假设未来24小时的温度等于当前温度
- 计算平均绝对误差（MAE）作为基准
- 后续模型需要超越这个基准才有意义

In [ ]:
def evaluate_naive_method():
    """
    评估朴素方法的性能
    
    朴素方法假设：未来温度 = 当前温度
    这是最简单的预测策略，作为性能基准
    
    Returns:
    --------
    float : 平均绝对误差（MAE）
    """
    batch_maes = []
    for step in range(val_steps):
        samples, targets = next(val_gen)
        # 使用序列最后一个时间步的温度作为预测值
        preds = samples[:, -1, 1]  # 温度特征在索引1
        mae = np.mean(np.abs(preds - targets))
        batch_maes.append(mae)
    return np.mean(batch_maes)

naive_mae = evaluate_naive_method()
print(f"朴素方法的MAE: {naive_mae:.4f}")
print(f"转换为摄氏度: {naive_mae * std[1]:.2f}°C")

## 8. 全连接网络（Baseline）

使用简单的全连接网络作为第一个学习模型。

**模型架构**：
- Flatten层：将3D输入展平为2D
- Dense层：32个神经元，ReLU激活
- Dense层：1个输出神经元（温度预测）

**特点**：
- 不考虑时间顺序，只是简单的模式匹配
- 作为深度学习的基准对比

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop

# 构建全连接网络模型
model = Sequential([
    layers.Input(shape=(lookback // step, float_data.shape[-1])),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
], name='DenseNet_Baseline')

# 编译模型
model.compile(
    optimizer=RMSprop(learning_rate=0.001),
    loss='mae',
    metrics=['mae']
)

# 显示模型结构
model.summary()

# 训练模型（使用较少的epochs进行测试）
history = model.fit(
    train_gen,
    steps_per_epoch=200,
    epochs=20,
    validation_data=val_gen,
    validation_steps=val_steps,
    verbose=1
)

In [ ]:
def plot_training_history(history, title='Training and Validation Loss', ylim=(0, 0.9)):
    """
    绘制训练和验证损失曲线
    
    Parameters:
    -----------
    history : History object
        Keras训练历史对象
    title : str
        图表标题
    ylim : tuple
        Y轴范围
    """
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(loss) + 1)
    
    plt.figure(figsize=(12, 5))
    plt.plot(epochs, loss, 'bo-', label='Training loss', alpha=0.7)
    plt.plot(epochs, val_loss, 'rs-', label='Validation loss', alpha=0.7)
    plt.title(title, fontsize=14, pad=15)
    plt.xlabel('Epochs', fontsize=12)
    plt.ylabel('Loss (MAE)', fontsize=12)
    plt.legend(fontsize=11)
    plt.ylim(ylim)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# 绘制全连接网络的训练历史
plot_training_history(history, title='Dense Network: Training and Validation Loss')

## 9. GRU循环网络

GRU（Gated Recurrent Unit）是一种循环神经网络，能够捕捉时间序列中的长期依赖关系。

**模型架构**：
- GRU层：32个单元
- Dense层：1个输出神经元

**优势**：
- 能够学习时间模式和趋势
- 比全连接网络更适合序列数据
- 参数量相对较少，训练效率高

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop

# 构建GRU模型
model = Sequential([
    layers.Input(shape=(None, float_data.shape[-1])),
    layers.GRU(32),
    layers.Dense(1)
], name='GRU_Model')

# 编译模型
model.compile(
    optimizer=RMSprop(learning_rate=0.001),
    loss='mae',
    metrics=['mae']
)

# 显示模型结构
model.summary()

# 训练模型
history = model.fit(
    train_gen,
    steps_per_epoch=500,
    epochs=20,
    validation_data=val_gen,
    validation_steps=val_steps,
    verbose=1
)

In [ ]:
# 绘制GRU模型的训练历史
plot_training_history(history, title='GRU Model: Training and Validation Loss')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop

# 构建带Dropout的GRU模型
model = Sequential([
    layers.Input(shape=(None, float_data.shape[-1])),
    layers.GRU(32, dropout=0.1, recurrent_dropout=0.2),
    layers.Dense(1)
], name='GRU_Dropout_Model')

# 编译模型
model.compile(
    optimizer=RMSprop(learning_rate=0.001),
    loss='mae',
    metrics=['mae']
)

# 显示模型结构
model.summary()

# 训练模型
history = model.fit(
    train_gen,
    steps_per_epoch=500,
    epochs=20,
    validation_data=val_gen,
    validation_steps=val_steps,
    verbose=1
)

## 10. 使用Dropout正则化的GRU模型

为了降低过拟合，引入Dropout正则化技术。

**Dropout类型**：
- **dropout**: 输入单元的dropout率
- **recurrent_dropout**: 循环连接的dropout率

**效果**：
- 防止模型过度依赖某些特征
- 提高模型的泛化能力

In [ ]:
# 绘制带Dropout的GRU模型训练历史
plot_training_history(history, title='GRU with Dropout: Training and Validation Loss')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop

# 构建堆叠GRU模型
model = Sequential([
    layers.Input(shape=(None, float_data.shape[-1])),
    layers.GRU(32, dropout=0.1, recurrent_dropout=0.2, return_sequences=True),
    layers.GRU(64, dropout=0.1, recurrent_dropout=0.5, activation='relu'),
    layers.Dense(1)
], name='Stacked_GRU_Model')

# 编译模型
model.compile(
    optimizer=RMSprop(learning_rate=0.001),
    loss='mae',
    metrics=['mae']
)

# 显示模型结构
model.summary()

# 训练模型
history = model.fit(
    train_gen,
    steps_per_epoch=500,
    epochs=40,
    validation_data=val_gen,
    validation_steps=val_steps,
    verbose=1
)

## 11. 堆叠GRU网络

通过堆叠多层GRU来增强模型的表示能力。

**模型架构**：
- GRU层1：32个单元，return_sequences=True（返回完整序列）
- GRU层2：64个单元
- Dense层：1个输出神经元

**关键点**：
- 第一层GRU需要设置`return_sequences=True`以传递序列给下一层
- 更深的网络可以学习更复杂的模式
- 增加dropout防止过拟合

In [ ]:
# 绘制堆叠GRU模型训练历史
plot_training_history(history, title='Stacked GRU Model: Training and Validation Loss')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop

# 构建LSTM模型
model = Sequential([
    layers.Input(shape=(None, float_data.shape[-1])),
    layers.LSTM(32, dropout=0.1, recurrent_dropout=0.2),
    layers.Dense(1)
], name='LSTM_Model')

# 编译模型
model.compile(
    optimizer=RMSprop(learning_rate=0.001),
    loss='mae',
    metrics=['mae']
)

# 显示模型结构
model.summary()

# 训练模型
history = model.fit(
    train_gen,
    steps_per_epoch=500,
    epochs=20,
    validation_data=val_gen,
    validation_steps=val_steps,
    verbose=1
)

## 12. LSTM网络用于温度预测

LSTM（Long Short-Term Memory）是另一种强大的循环神经网络，专门设计用于处理长期依赖问题。

**LSTM vs GRU**：
- LSTM有更复杂的门控机制（输入门、遗忘门、输出门）
- GRU更简单，参数更少，训练更快
- 在某些任务上LSTM可能表现更好，需要实验对比

**模型架构**：
- LSTM层：32个单元
- Dense层：1个输出神经元

## 13. 双向GRU网络

双向RNN能够同时从两个方向处理序列：从过去到未来，以及从未来到过去。

**双向网络的优势**：
- 对于某些模式，未来的信息也很重要
- 能够捕获更全面的时间依赖关系
- 在序列标注、翻译等任务中效果显著

**注意**：
- 双向网络不能用于实时预测（需要看到完整序列）
- 参数量是单向网络的两倍
- 训练时间更长

**模型架构**：
- 双向GRU层：32个单元（正向和反向各32个）
- Dense层：1个输出神经元

In [ ]:
# 绘制LSTM模型训练历史
plot_training_history(history, title='LSTM Model: Training and Validation Loss')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop

# 构建双向GRU模型
model = Sequential([
    layers.Input(shape=(None, float_data.shape[-1])),
    layers.Bidirectional(layers.GRU(32, dropout=0.1, recurrent_dropout=0.2)),
    layers.Dense(1)
], name='Bidirectional_GRU_Model')

# 编译模型
model.compile(
    optimizer=RMSprop(learning_rate=0.001),
    loss='mae',
    metrics=['mae']
)

# 显示模型结构
model.summary()

# 训练模型
history = model.fit(
    train_gen,
    steps_per_epoch=500,
    epochs=40,
    validation_data=val_gen,
    validation_steps=val_steps,
    verbose=1
)

In [ ]:
# 绘制双向GRU模型训练历史
plot_training_history(history, title='Bidirectional GRU Model: Training and Validation Loss')

## 14. 总结与最佳实践

### 模型性能对比

通过本notebook，我们实现并比较了多种时间序列预测模型：

1. **朴素方法**: 基准模型，MAE约为0.34
2. **全连接网络**: 简单的深度学习baseline
3. **GRU网络**: 单层循环网络
4. **GRU + Dropout**: 添加正则化，减少过拟合
5. **堆叠GRU**: 多层网络，增强表示能力
6. **LSTM网络**: 另一种循环网络架构
7. **双向GRU**: 同时利用过去和未来信息

### 关键要点

**数据预处理**：
- 使用训练集的统计量进行标准化，避免数据泄露
- 合理划分训练/验证/测试集
- 使用生成器处理大规模数据

**模型设计**：
- 从简单模型开始，逐步增加复杂度
- 使用Dropout防止过拟合
- 堆叠多层可以提升性能，但要注意过拟合风险
- 双向网络适合离线分析，不适合实时预测

**训练技巧**：
- 监控验证集性能，及时停止训练
- 使用合适的学习率
- 根据任务选择合适的批次大小

### 进一步改进方向

1. **早停机制**: 使用EarlyStopping回调防止过拟合
2. **学习率调度**: 动态调整学习率
3. **更多特征工程**: 添加时间特征（星期、月份等）
4. **注意力机制**: 让模型关注重要的时间步
5. **集成方法**: 结合多个模型的预测结果